In [1]:
import pandas as pd
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing as preproc
from scipy import stats
import numpy as np

In [2]:
df=pd.read_parquet("diabetic_data_cleaned_pca_som_diag_grouped_ae.parquet")

In [3]:
df.head()

,encounter_id,patient_nbr,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,...,ae_3,ae_4,ae_5,ae_6,ae_7,ae_8,ae_9,ae_10,ae_11,ae_12
0,149190,55629189,Caucasian,Female,[10-20),1,1,7,3,59,...,16.940006,23.097187,19.296238,29.459946,30.113405,40.322517,18.857946,15.380990,23.321814,27.334114
1,64410,86047875,AfricanAmerican,Female,[20-30),1,1,7,2,11,...,33.362064,16.557415,10.222631,16.109257,14.378604,32.578545,21.602501,3.653939,28.548956,37.455536
2,500364,82442376,Caucasian,Male,[30-40),1,1,7,2,44,...,22.848877,40.628677,40.006958,54.089451,23.360697,35.270077,28.743443,14.732458,37.284184,24.839588
3,16680,42519267,Caucasian,Male,[40-50),1,1,7,1,51,...,35.784733,30.247353,50.454319,43.604866,35.832771,39.827732,36.632324,13.657840,39.127926,30.951803
4,35754,82637451,Caucasian,Male,[50-60),2,1,2,3,31,...,23.921968,27.939154,41.540062,34.101780,34.929150,13.991295,9.326398,14.794641,36.444004,44.284451


In [4]:
df.columns, df.shape

(Index(['encounter_id', 'patient_nbr', 'race', 'gender', 'age',
        'admission_type_id', 'discharge_disposition_id', 'admission_source_id',
        'time_in_hospital', 'num_lab_procedures', 'num_procedures',
        'num_medications', 'number_outpatient', 'number_emergency',
        'number_inpatient', 'diag_1', 'diag_2', 'diag_3', 'number_diagnoses',
        'max_glu_serum', 'A1Cresult', 'metformin', 'repaglinide', 'glimepiride',
        'glipizide', 'glyburide', 'pioglitazone', 'rosiglitazone', 'insulin',
        'glyburide-metformin', 'change', 'diabetesMed', 'readmitted', 'PC1',
        'PC2', 'PC3', 'diag_1_group', 'diag_2_group', 'diag_3_group',
        'som_jaccard_0', 'som_jaccard_1', 'som_euc_0', 'som_euc_1', 'ae_0',
        'ae_1', 'ae_2', 'ae_3', 'ae_4', 'ae_5', 'ae_6', 'ae_7', 'ae_8', 'ae_9',
        'ae_10', 'ae_11', 'ae_12'],
       dtype='object'),
 (100244, 56))

In [5]:
DB_COLUMNS = [
  'PC1','PC2', 'PC3',
  'ae_0', 'ae_1', 'ae_2', 'ae_3', 'ae_4',
  'ae_5', 'ae_6', 'ae_7', 'ae_8', 'ae_9',
  'ae_10', 'ae_11', 'ae_12',
  # 'ae_13', 'ae_14',
  # 'ae_15', 'ae_16', 'ae_17'
]

df[DB_COLUMNS]

,PC1,PC2,PC3,ae_0,ae_1,ae_2,ae_3,ae_4,ae_5,ae_6,ae_7,ae_8,ae_9,ae_10,ae_11,ae_12
0,-0.214843,0.070744,-0.194516,30.020775,23.524881,21.868984,16.940006,23.097187,19.296238,29.459946,30.113405,40.322517,18.857946,15.380990,23.321814,27.334114
1,0.417941,-0.522845,0.160733,26.739025,22.506536,24.514652,33.362064,16.557415,10.222631,16.109257,14.378604,32.578545,21.602501,3.653939,28.548956,37.455536
2,-0.129162,-0.131116,-0.050765,32.675808,20.616684,15.409163,22.848877,40.628677,40.006958,54.089451,23.360697,35.270077,28.743443,14.732458,37.284184,24.839588
3,-0.333245,-0.143083,-0.009183,35.431171,40.692886,35.721672,35.784733,30.247353,50.454319,43.604866,35.832771,39.827732,36.632324,13.657840,39.127926,30.951803
4,0.640729,-0.446888,-0.059434,24.872639,18.126051,26.878084,23.921968,27.939154,41.540062,34.101780,34.929150,13.991295,9.326398,14.794641,36.444004,44.284451
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100239,-0.226515,0.049629,-0.151316,18.564442,23.728540,11.443975,33.054741,25.154684,30.259825,16.011316,34.233112,29.103378,35.363621,41.357502,28.804075,2.872444
100240,0.266458,-0.086568,-0.008378,44.234550,16.063749,29.040533,39.558475,25.493149,23.887850,32.374828,25.373158,34.035446,26.360598,35.005196,29.873354,24.153151
100241,-0.277035,-0.027778,-0.406056,14.694986,17.261377,10.221172,9.171141,23.729177,33.675728,10.703976,22.745504,28.254274,22.469522,18.784910,21.129669,8.416029
100242,0.291241,0.333055,0.077571,20.603630,20.004524,17.100962,28.563631,26.978170,14.465246,22.566378,17.393518,17.935911,9.871636,15.989691,7.418999,12.163943


In [6]:
X=df[DB_COLUMNS]

# Scale the independent variables
#scaler = StandardScaler()
X_scaled = preproc.MinMaxScaler().fit_transform(X)

In [7]:
# Apply DBSCAN
epsilon = 0.39 # You may need to adjust this parameter
min_samples = 15 # You may need to adjust this parameter
dbscan = DBSCAN(eps=epsilon, min_samples=min_samples, n_jobs=-1)
#dbscan.fit(X_scaled)
dbscan.fit(X_scaled)

DBSCAN(eps=0.39, min_samples=15, n_jobs=-1)

In [8]:
# Extract outlier labels
outlier_mask = dbscan.labels_ == -1

# Get the outliers
outliers = X[outlier_mask]

print(len(outliers))

988


In [9]:
len(set(dbscan.labels_))

2

In [10]:
# Analyze outliers
print(len(df[outlier_mask]))

988


In [11]:
non_outlier_mask=dbscan.labels_ != -1

In [12]:
print(len(df[non_outlier_mask]))

99256


In [13]:
def display_df(df: pd.DataFrame, title: str = ""):
  display(df.style.hide(axis="index").set_caption(title))

In [14]:
TGT = "readmitted"

In [15]:
prop_total = df.loc[:, TGT].value_counts().reset_index()
display_df(prop_total.sort_values("readmitted", ascending = False), "full dataset")

prop_outs = df.loc[list(outlier_mask), TGT].value_counts().reset_index()
display_df(prop_outs.sort_values("readmitted", ascending = False), "DBscan outliers")

stats.chi2_contingency(np.array([prop_total["count"].values, prop_outs["count"].values]))

readmitted,count
NO,53821
>30,35173
<30,11250


readmitted,count
NO,541
>30,349
<30,98


Chi2ContingencyResult(statistic=1.706170869903234, pvalue=0.4260982033636429, dof=2, expected_freq=array([[53831.43993994, 35175.31381381, 11237.24624625],
       [  530.56006006,   346.68618619,   110.75375375]]))

In [16]:
output_df=df[non_outlier_mask]

In [17]:
filename = "diabetic_data_cleaned_pca_data_enriched_som_ae_dbscan"
output_df.to_csv(f"{filename}.csv")
output_df.to_parquet(f"{filename}.parquet")